In [21]:
from pulp import *
import numpy as np
MEPS=1.0e-8

In [22]:
class KnapsackProblem(object):
    """The difinition of KnapSackProblem """
    def __init__(self,name,capacity,items,costs,weights,zeros=set(),ones=set()):
        self.name=name
        self.capacity=capacity
        self.items=items
        self.costs=costs
        self.weights=weights
        self.zeros=zeros
        self.ones=ones
        self.lb=-100
        self.ub=-100
        ratio={j:costs[j]/weights[j] for j in items}
        self.sitemList=[k for k, 
                        v in sorted(ratio.items(),key=lambda x:x[1], reverse=True)]
        self.xlb={j:0 for j in self.items}
        self.xub={j:0 for j in self.items}
        self.bi=None
        
    def getbounds(self):
        """ Calculate the upper and lower bounds. """
        for j in self.zeros:
            self.xlb[j]=self.xub[j]=0
        for j in self.ones:
            self.xlb[j]=self.xub[j]=1
        if self.capacity < sum(self.weights[j] for j in self.ones):
            self.lb=self.ub=-100
            return 0
        
        ritems = self.items-self.zeros-self.ones
        sitems=[j for j in self.sitemList if j in ritems]
        cap=self.capacity-sum(self.weights[j] for j in self.ones)
        
        for j in sitems:
            if self.weights[j] <= cap:
                cap-=self.weights[j]
                self.xlb[j]=self.xub[j]=1
            else:
                self.xub[j]=cap/self.weights[j]
                self.bi=j
                break
            self.lb=sum(self.costs[j]*self.xlb[j] for j in self.items)
            self.ub=sum(self.costs[j]*self.xub[j] for j in self.items)
        
    def __str__(self):
        """ KnapSachProblemの情報を印字 """
        return ('Nama = '+self.name+',capacity='+str(self.capacity)+',\n'
                'items = '+str(self.items)+',\n'+
                'costs = '+str(self.costs)+',\n'+
                'weights = '+str(self.weights)+',\n'+
                'zeros = '+str(self.zeros)+', ones = '+str(self.ones)+',\n'+
                'lb = '+str(self.lb)+', ub = '+str(self.ub)+',\n'+
                'sitemList = '+str(self.sitemList)+',\n'+
                'xlb = '+str(self.xlb)+',\n'+'xub = '+str(self.xub)+',\n'+
                'bi = '+str(self.bi)+'\n')

In [23]:
def KnapsackProblemSolve(capacity,items,costs,weights):
    from collections import deque
    queue=deque()
    root=KnapsackProblem('KP',capacity=capacity,
                         items=items,costs=costs,weights=weights,
                         zeros=set(),ones=set())
    root.getbounds()
    best=root
    queue.append(root)
    
    while queue !=deque([]):
        p=queue.popleft()
        p.getbounds()
        if p.ub> best.lb: #bestを更新する可能性がある
            if p.lb>best.lb: #bestを更新する
                best=p
            if p.ub>p.lb: #子問題を作って分枝する
                k=p.bi
                p1=KnapsackProblem(p.name+'+'+str(k),
                                   capacity=p.capacity,items=p.items,
                                   costs=p.costs,weights=p.weights,
                                   zeros=p.zeros,ones=p.ones.union({k}))
                queue.append(p1)
                p2=KnapsackProblem(p.name+'-'+str(k),
                                   capacity=p.capacity,items=p.items,
                                   costs=p.costs,weights=p.weights,
                                   zeros=p.zeros.union({k}),ones=p.ones)
                queue.append(p2)
    return 'Optimal', best.lb, best.xlb

In [24]:
def binpacking2(capacity,w):
    m=len(w)
    items=set(range(m))
    A=np.identity(m)
    
    solved=False
    columns=0
    dual=LpProblem(name='D(K)', sense=LpMaximize)
    y=[LpVariable('y'+str(i),lowBound=0) for i in items]
    
    dual += lpSum(y[i] for i in items) # 目的関数の設定
    for j in range(len(A.T)): # 制約条件の付加
        dual += lpDot(A.T[j],y) <=1, 'ineq'+str(j)
        
    while not(solved):
        #dual
        dual.solve()
        
        costs={i: y[i].varValue for i in items}
        weights={i: w[i] for i in items}
        (state, val, sol) = KnapsackProblemSolve(capacity, items, costs,weights)
        
        if val >= 1.0+MEPS:
            a=np.array([int(sol[i]) for i in items])
            dual += lpDot(a,y) <= 1 ,'ineq'+str(m+columns)
            A=np.hstack((A,a.reshape((-1,1))))
            columns+=1
        else:
            solved=True
            
    print('Generated columns: ', columns)
    
    m,n =A.shape
    primal =LpProblem(name='P(K)',sense=LpMinimize)
    x =[LpVariable('x'+str(j),lowBound=0,cat='Binary') for j in range(n)]
    
    primal += lpSum(x[j] for j in range(n)) # 目的関数の設定
    for i in range(m): # 制約条件の付加
        primal += lpDot(A[i],x) >=1,'ineq'+str(i)
    
    primal.solve()
    if value(primal.objective)-value(dual.objective) < 1.0-MEPS:
        print('Optimal solution found: ')
    else:
        print('Approximated solution found: ')
    K =[j for j in range(n) if x[j].varValue > MEPS]
    result =[]
    itms =set(range(m))
    for j in K:
        J = {i for i in range(m) if A[i,j] > MEPS and i in items}
        r = [w[i] for i in J]
        items -= J
        result.append(r)
    print(result)       

In [47]:
capacity = 25
items = set(range(60))
np.random.seed(2)
w={i:np.random.randint(5,10) for i in items}
w2= [w[i] for i in items]
print(w2)

%time binpacking2(capacity,w)

[5, 5, 8, 7, 8, 5, 7, 6, 8, 7, 9, 9, 9, 8, 9, 7, 8, 8, 7, 6, 7, 9, 8, 5, 9, 8, 6, 7, 5, 9, 9, 7, 9, 7, 6, 5, 7, 7, 6, 5, 6, 5, 7, 6, 6, 6, 9, 7, 8, 5, 8, 5, 7, 7, 5, 9, 7, 5, 7, 9]
Generated columns:  96
Approximated solution found: 
[[5, 5, 5, 5, 5], [5, 5, 5, 5], [6, 6, 6, 6], [6, 6, 6, 6], [7, 7, 7], [7, 7], [7, 7], [7, 7, 7], [7, 7], [7, 7], [8, 8, 8], [8, 8], [8, 8], [8, 8, 8], [5, 9], [9], [5, 9, 6], [7, 9, 9], [7, 9], [9, 9, 7], [9, 9], [9, 9, 5]]
CPU times: user 594 ms, sys: 320 ms, total: 915 ms
Wall time: 1.91 s


In [13]:
def KPS(capacity, items,costs,weights):
    knapsack=LpProblem(name='knapsack', sense=LpMaximize)
    x={j:LpVariable('x'+str(j),lowBound=0,cat='binary') for j in items}
    
    knapsack += lpSum(costs[j]*x[j] for j in items) # 目的関数の設定
    knapsack += lpSum(weights[j]*x[j] for j in items) <= capacity, 'weights'
    
    knapsack.solve()
    xx={j:int(x[j].varValue) for j in items}
    return LpStatus[knapsack.status], value(knapsack.objective),xx